<a href="https://colab.research.google.com/github/SolutionLr/DL/blob/main/Coffee_Beans_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import necessary modules 
from google.colab import drive
drive.mount('/content/drive')
from PIL import Image
from torch.utils.data import Dataset
from skimage import io
from sklearn.model_selection import train_test_split
from torchvision.models import resnet34
from coffee_beans_dataset import Coffee_Beans
import pandas as pd
import matplotlib.pyplot as plt
import torch
import numpy as np
import torch.nn as nn
import torchvision.transforms as transforms
import os
import torchvision
torch.manual_seed(0)


#Create model
class Resnet(nn.Module):
    def __init__(self):
        super(Resnet, self).__init__()
        self.resnet = resnet34(pretrained=True)
        for param in self.resnet.parameters():
            param.requires_grad = False
        self.resnet.fc = nn.Sequential(nn.Linear(512, 4))

    def forward(self, x):
        pred = self.resnet(x)
        return pred


def show_predict(model, loader):
    batch = next(iter(loader))
    images, _ = batch
    images = images.to(device)
    pred = model(images)
    grid = torchvision.utils.make_grid(images[0:6].cpu(), nrow=3)
    plt.figure(figsize=(11, 11))
    plt.imshow(np.transpose(grid, (1, 2, 0)))
    print(pred[0:6].argmax(1))
    plt.show()
    plt.close()


#Load data
root_dir = '/content/drive/MyDrive/DL_1/coffee_beans_classifier/Coffee'
csv_file = '/content/drive/MyDrive/DL_1/coffee_beans_classifier/Coffee/Coffee Bean.csv'
coffee_beans_dataset = Coffee_Beans(root_dir, csv_file, transform=transforms.ToTensor())
train_loader, test_loader = coffee_beans_dataset.train_test_loader()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = Resnet()
model.to(device)
loss = nn.CrossEntropyLoss()
epoch = 3
lr = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=lr)


#Create train and test loops
def train_loop(train_loader, loss, optimizer, model):
    size_dataset = len(train_loader.dataset)
    model.train()
    for X, y in train_loader:
        correct = 0
        X = X.to(device)
        y = y.to(device)
        y = torch.nn.functional.one_hot(y)
        pred = model(X)
        L = loss(pred, y.argmax(1))
        optimizer.zero_grad()
        L.backward()
        optimizer.step()
        correct = (pred.argmax(1) == y.argmax(1)).float().sum()
        accuracy = (correct / len(y)) * 100
        print(f"train_loss: {L.item()}, accuracy/batch: {accuracy}")


def test_loop(test_loader, loss, model):
    model.eval()
    size_dataset = len(test_loader.dataset)
    correct, test_loss = 0, 0
    with torch.no_grad():
        for X, y in test_loader:
            X = X.to(device)
            y = y.to(device)
            y = torch.nn.functional.one_hot(y)
            y = y.argmax(1)
            pred = model(X)
            test_loss += loss(pred, y).item() * X.shape[0]
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        test_loss = (test_loss / size_dataset)
        accuracy = (correct / size_dataset) * 100
        print(f"test_loss: {test_loss}, accuracy/epoch: {accuracy}")


for i in range(epoch):
    train_loop(train_loader, loss, optimizer, model)
    test_loop(test_loader, loss, model)
torch.save(model.state_dict(), '/content/drive/MyDrive/DL_1/coffee_beans_classifier/ResNet.py')

# show some predicted values
show_predict(model, test_loader)
